In [15]:
import re
import unidecode
import itertools
from nltk import ngrams
import string
import numpy as np
from tqdm import tqdm
import os
import torch
from dataloader.dataset import BasicDataset, Collator
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import torch.nn as nn
import torch.nn.functional as F
from config import alphabet
from optim.loss import LabelSmoothingLoss

## Create model

In [2]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout):
        super().__init__()

        self.rnn = nn.GRU(hidden_size, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        self.embedding = nn.Embedding(input_size, hidden_size)

    def forward(self, src):
        """
        src: batch_size x time_step x number_class
        outputs: batch_size x max_length x enc_hid_size ** 2
        hidden: batch_size x hid_dim
        """
        embedded = self.embedding(src)
        embedded = embedded.permute(1, 0, 2)
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)))
        
        return outputs, hidden

In [3]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        """
        hidden: batch_size x hid_dim
        encoder_outputs: src_len x batch_size x hid_dim,
        outputs: batch_size x src_len
        """
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)

        return F.softmax(attention, dim=1)

In [4]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attention = Attention(hidden_size, hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size * 3, self.hidden_size)
        self.fc_out = nn.Linear(hidden_size * 4, output_size)

    def forward(self, inputs, hidden, encoder_outputs):
        inputs = inputs.unsqueeze(0)
        embedded = self.embedding(inputs)
        embedded = self.dropout(embedded)
        
        # caculate attention weight
        attn_weights = self.attention(hidden, encoder_outputs)
        attn_weights = attn_weights.unsqueeze(1)
        
        # apply attention weight to encoder output
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        attn_applied = torch.bmm(attn_weights, encoder_outputs)
        attn_applied = attn_applied.permute(1, 0, 2)
        
        # decoder
        rnn_input = torch.cat((embedded, attn_applied), 2)
        output, hidden = self.gru(rnn_input, hidden.unsqueeze(0))
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        attn_applied = attn_applied.squeeze(0)
        
        fc_input = torch.cat((output, attn_applied, embedded), dim=1)
        prediction = self.fc_out(fc_input)
        
        return prediction, hidden.squeeze(0), attn_weights

In [5]:
class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, encoder_hidden, decoder_hidden, dropout=0.1):
        super().__init__()
        self.encoder = Encoder(vocab_size, encoder_hidden, dropout)
        self.decoder = AttnDecoderRNN(decoder_hidden, vocab_size)

    def forward_encoder(self, src):
        """
        src: timestep x batch_size x channel
        hidden: batch_size x hid_dim
        encoder_outputs: src_len x batch_size x hid_dim
        """

        encoder_outputs, hidden = self.encoder(src)

        return hidden, encoder_outputs

    def forward_decoder(self, tgt, memory):
        """
        tgt: timestep x batch_size
        hidden: batch_size x hid_dim
        encoder: src_len x batch_size x hid_dim
        output: batch_size x 1 x vocab_size
        """

        tgt = tgt[-1]
        hidden, encoder_outputs = memory
        output, hidden, _ = self.decoder(tgt, hidden, encoder_outputs)
        output = output.unsqueeze(1)

        return output, (hidden, encoder_outputs)

    def forward(self, src, trg):
        """
        src: time_step x batch_size
        trg: time_step x batch_size
        outputs: batch_size x time_step x vocab_size
        """
        batch_size = src.shape[0]
        trg_len = src.shape[1]
        trg_vocab_size = self.decoder.output_size
        device = src.device

        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        encoder_outputs, hidden = self.encoder(src)
        for t in range(trg_len):
            input = trg[t]
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            
        outputs = outputs.transpose(0, 1).contiguous()
        return outputs

In [6]:
model = Seq2Seq(len(alphabet), encoder_hidden=256, decoder_hidden=256)

## Define parameters

In [7]:
batch_size = 32
valid_every = 1000
print_every = 200
lr = 0.01
num_iters = 100000
device = ("cuda:1" if torch.cuda.is_available() else "cpu")

## Define loss function

In [8]:
from optim.loss import LabelSmoothingLoss
criterion = LabelSmoothingLoss(len(alphabet), 0).cpu()
optimizer = AdamW(model.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-09)
scheduler = OneCycleLR(optimizer, max_lr=lr, total_steps=num_iters, pct_start=0.1)

## Create train and valid data

In [9]:
dataset = BasicDataset()

100%|██████████| 5478334/5478334 [01:05<00:00, 84172.14it/s]


In [10]:
from torch.utils.data import DataLoader, random_split

# split train and val dataloader
split_ratio = 0.99
n_train = int(len(dataset) * split_ratio)
n_val = len(dataset) - n_train
train_dataset, val_dataset = random_split(dataset, [n_train, n_val])

In [11]:
print('The number of train data: ', n_train)
print('The number of val data: ', n_val)

The number of train data:  19566671
The number of val data:  197644


In [12]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=Collator(), shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=Collator(), shuffle=False, num_workers=8, pin_memory=True, drop_last=True)
data_iter = iter(train_loader)

## Training

In [13]:
model = model.to(device)

In [14]:
from utils import batch_to_device, cal_acc

In [15]:
def validate():
    model.eval()
    total_loss = []
    full_seq_acc_list = []
    chars_acc_list = []
    val_loss = 0,
    val_acc = 0,
    with torch.no_grad():
        for batch in val_loader:
            texts, tgt_input, tgt_output = batch
            texts, tgt_input, tgt_output = batch_to_device(texts, tgt_input, tgt_output, device)
            outputs = model(texts, tgt_input)
            outputs = outputs.flatten(0, 1)
            tgt_output = tgt_output.flatten()
            loss = criterion(outputs, tgt_output)
            # full_seq_acc, char_acc = cal_acc(outputs, tgt_input)
            total_loss.append(loss.item())
            # full_seq_acc_list.append(full_seq_acc)
            # chars_acc_list.append(char_acc)
            del outputs,
            del loss,
            
    val_loss = np.mean(total_loss)
#     full_seq_acc = np.mean(full_seq_acc_list)
#     char_acc = np.mean(chars_acc_list)
    model.train()
    
    
#     return val_loss, full_seq_acc, char_acc
    return val_loss

In [16]:
def train_step(batch):
    # get the inputs
    texts, tgt_input, tgt_output = batch
    texts, tgt_input, tgt_output = batch_to_device(texts, tgt_input, tgt_output, device)
    # zero the parameter gradients
    optimizer.zero_grad()
    
    # forward + backward + optimize + scheduler
    outputs = model(texts, tgt_input)
    outputs = outputs.flatten(0, 1)
    tgt_output = tgt_output.flatten()
    loss = criterion(outputs, tgt_output)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    optimizer.step()
    scheduler.step()
    
    loss_item = loss.item()
    
    return loss_item

In [17]:
import time

total_loss = 0
best_acc = 0
best_loss = 1000
global_step = 0
weight_path = 'weight.pth'

for i in range(num_iters):
    model.train()
    
    try:
        batch = next(data_iter)
    except StopIteration:
        data_iter = iter(train_loader)
        batch = next(data_iter)
    
    texts, tgt_input, tgt_output = batch
    global_step += 1
    start = time.time()
    try:
        loss = train_step(batch)
    except Exception:
        print('max: ', tgt_input.cpu().numpy().max())
    end = time.time()
    total_loss += loss

    if global_step % print_every == 0:
        print('step: {:06d}, train_loss: {:.4f}, gpu_time: {}'.format(global_step, total_loss / print_every, end - start))
        total_loss = 0
        

    if global_step % valid_every == 0:
        # validate 
#         val_loss, full_seq_acc, char_acc = validate()
        val_loss = validate()
        print('================================')
        print('val loss: ', val_loss)
        if best_loss > val_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), weight_path)
            
        print("==============================================================================")
        # print("val_loss: {:.4f}, full_seq_acc: {:.4f}, char_acc: {:.4f}".format(val_loss, full_seq_acc, char_acc))
        print("val_loss: {:.4f}".format(val_loss))
        print("==============================================================================")

step: 000200, train_loss: 1.8131, gpu_time: 0.532719612121582
step: 000400, train_loss: 0.5177, gpu_time: 0.5884552001953125
step: 000600, train_loss: 0.2962, gpu_time: 0.716571569442749
step: 000800, train_loss: 0.2535, gpu_time: 0.6035690307617188
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/home/manhbui/anaconda3/envs/manhbq/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-d3b0f1a1e4c5>", line 22, in <module>
    loss = train_step(batch)
  File "<ipython-input-16-6baebdb568ed>", line 13, in train_step
    loss.backward()
  File "/home/manhbui/anaconda3/envs/manhbq/lib/python3.7/site-packages/torch/tensor.py", line 198, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/manhbui/anaconda3/envs/manhbq/lib/python3.7/site-p

TypeError: object of type 'NoneType' has no len()

In [ ]:
# Inference

In [56]:
file = open('corpus-full.txt', 'r')

In [57]:
data = file.readlines()

In [58]:
data[-10:]

['Mỗi cặp chỉ đẻ 1 quả trứng mỗi năm, có khi 2 năm, và 90% con non thường chết trong năm đầu tiên.Vì vậy, nhằm phục hồi số lượng kền kền, chìa khóa thành công bao gồm việc thuyết phục mọi người rằng đây là lợi ích lâu dài của chính họ và bảo vệ các con trưởng thành khỏi nguồn chất độc.\xa0Các đơn vị bảo tồn trên khắp châu Phi đang phối hợp với nhau, tập trung tập huấn cho lực lượng chức năng và kiểm lâm, nhanh chóng loại bỏ các xác động vật bị nhiễm độc - vốn không phải là chuyện dễ giữa một diện tích rộng lớn như vậy.Các chương trình giáo dục cho cộng đồng nông thôn và thúc đẩy thay đổi cơ sở hạ tầng là những cách tiếp cận dài hạn.\xa0Tổ chức Bảo tồn Nigeria đang làm việc cùng hơn 80 "thầy lang" địa phương để thay đổi quan niệm dùng kền kền chữa bệnh\n',
 'Tất cả các quốc gia châu Phi có kền kền sinh sống đã thống nhất một kế hoạch hành động lâu dài để bảo tồn loài chim này, trong đó có hẳn một lộ trình cho 12 năm tới.\xa0Học hỏi các kế hoạch tương tự ở châu Á, các tổ chức bảo tồn miề

In [59]:
len(data)

112921289

In [60]:
new_data = list(set(data))

In [62]:
len(data) - len(new_data)

305067

In [64]:
file = open('train_data.txt', 'w')
file.writelines(new_data)

In [65]:
len(new_data)

112616222